In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../')

In [2]:
from networks.inception import InceptionResnet
from networks.simple_cnn import BaselineCNNClassifier
from networks.resnet_big import SupCEResNet
import torch
from thop import profile

In [3]:
import time

def measure_time(model, device, num_trial = 10):
    model = model.to(device=device)
    total_time = 0
    for i in range(num_trial):
        x = torch.rand((1, 1, 29, 29), device=device)
        start_time = time.time()
        out = model(x)
        total_time += time.time() - start_time
    return total_time*1000/num_trial

In [4]:
baseline_model = SupCEResNet(num_classes=5)

In [5]:
measure_time(baseline_model, device='cpu', num_trial=1000)

65.73471879959106

In [6]:
measure_time(baseline_model, device='cuda', num_trial=1000)

5.093194961547852

In [7]:
incep = InceptionResnet(n_classes=5)

In [8]:
measure_time(incep, device='cpu', num_trial=1000)

/home/dhkim1/miniconda3/envs/torch/lib/python3.7/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448224956/work/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


KeyboardInterrupt: 

In [ ]:
measure_time(incep, device= 'cuda', num_trial=1000)

In [9]:
x = torch.rand((1, 1, 29, 29), device='cpu')
baseline_model = baseline_model.to(device='cpu')
macs, params = profile(baseline_model, inputs=(x, ))
print('MACs (G): ', macs/1000**3)
print('Params (M): ', params/1000**2)

[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register count_bn() for <class 'torch.nn.modules.batchnorm.BatchNorm2d'>.
[WARN] Cannot find rule for <class 'torch.nn.modules.container.Sequential'>. Treat it as zero Macs and zero Params.
[WARN] Cannot find rule for <class 'networks.resnet_big.BasicBlock'>. Treat it as zero Macs and zero Params.
[INFO] Register count_adap_avgpool() for <class 'torch.nn.modules.pooling.AdaptiveAvgPool2d'>.
[WARN] Cannot find rule for <class 'networks.resnet_big.ResNet'>. Treat it as zero Macs and zero Params.
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
[WARN] Cannot find rule for <class 'networks.resnet_big.SupCEResNet'>. Treat it as zero Macs and zero Params.
MACs (G):  0.032292592
Params (M):  0.700533


/home/dhkim1/miniconda3/envs/torch/lib/python3.7/site-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448224956/work/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)


In [10]:
x = torch.rand((1, 1, 29, 29), device='cpu')
incep = incep.to(device='cpu')
macs, params = profile(incep, inputs=(x, ))
print('MACs (G): ', macs/1000**3)
print('Params (M): ', params/1000**2)

[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.pooling.MaxPool2d'>.
[WARN] Cannot find rule for <class 'torch.nn.modules.container.Sequential'>. Treat it as zero Macs and zero Params.
[WARN] Cannot find rule for <class 'networks.inception.Stem'>. Treat it as zero Macs and zero Params.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU'>.
[WARN] Cannot find rule for <class 'networks.inception.InceptionresenetA'>. Treat it as zero Macs and zero Params.
[WARN] Cannot find rule for <class 'networks.inception.ReductionA'>. Treat it as zero Macs and zero Params.
[WARN] Cannot find rule for <class 'networks.inception.InceptionresnetB'>. Treat it as zero Macs and zero Params.
[WARN] Cannot find rule for <class 'networks.inception.ReductionB'>. Treat it as zero Macs and zero Params.
[INFO] Register count_adap_avgpool() for <class 'torch.nn.modules.pooling.AdaptiveAvgPool2d'>.
[WARN] Cannot f